In [16]:
import pandas as  pd

import datetime as dt 
import numpy as np
from numpy.random import *

st  = dt.datetime(2019,1,1, 9, 0)
ed  = dt.datetime(2019,1,31, 9, 0)

ls_dt = []
lapse = st
while True:
    if lapse > ed:
        break
    ls_dt.append(lapse)
    lapse += dt.timedelta(minutes=30)

ls_code = range(1000,2000)

ls_volumeRatio = rand(1000)# for _ in range(10)]

df_datetime = pd.DataFrame(ls_dt, columns=['datetime'])
df_code = pd.DataFrame(ls_code, columns=['code'])
df_vr = pd.DataFrame(ls_volumeRatio, columns=['volumeRatio'])

df = pd.concat([df_datetime, df_code], ignore_index=True)

ls_df = []
for _code in ls_code:
#     print(_code)
    df_datetime = pd.DataFrame(ls_dt, columns=['datetime'])
    df_datetime['volumeRatio'] = rand(df_datetime.shape[0])
    df_datetime['code'] = _code
    ls_df.append(df_datetime)
df = pd.concat(ls_df, axis = 0, ignore_index=True)

# set(df.code)

df  = df.set_index('datetime')

df = df.loc[dt.time(9, 0): dt.time(15,0)]

df = df.reset_index()

In [17]:
df['dummyA'] = rand(df.shape[0])
df['dummyB'] = rand(df.shape[0])

In [18]:
df['catDummyA'] = np.round(rand(df.shape[0])*10).astype(int)
df['catDummyB'] = np.round(rand(df.shape[0])*5).astype(int)

In [22]:
df['codeSurlusA'] = df.code%7
df['codeSurlusB'] = df.code%10

In [36]:
df['weekday'] = df.datetime.dt.strftime('%w')
df['dateSpike'] = df.datetime.dt.strftime('%d').astype(int)//11

In [37]:
df.to_csv("dummy_vol.csv",index=False)

In [316]:
import scipy.sparse as sp
from sklearn.utils import indexable
from sklearn.utils.validation import _num_samples, column_or_1d
class customTimeSeriesSplit(TimeSeriesSplit):
    def __init__(self, n_splits=None, train_size=None, test_size=None, split_type = "flat",min_train_size=0):
        super().__init__(n_splits)#, max_train_size=None)
        self.min_train_size = min_train_size 
        self.train_size = train_size 
        self.test_size = test_size 
        self.n_splits = n_splits
        self.split_type = split_type 
        
        if self.split_type == 'flat':
            assert not self.n_splits is None , "n_splits must be set"
            assert not self.train_size is None,"train_size must be set"
            assert not self.test_size is None, "test_size must be set"
            if not self.min_train_size is None:
                print("min_train_size is ignored")
    
        if self.split_type  == 'with_min':
            assert not self.n_splits is None , "n_splits must be set"
            assert not self.test_size is None,"train_size must be set"
            assert not self.min_train_size is None, "min_train_size must be set"
            if not self.train_size is None:
                print("train_size is ignored")
        
    def get_parameter(self):
        print("train_size:{0}".format(self.train_size))
        print("test_size:{0}".format(self.test_size))
        print("n_splits:{0}".format(self.n_splits))
        print("min_train_size:{0}".format(self.min_train_size))
        print("split_type:{0}".format(self.split_type))
        
    def get_splitIndex(self,X):
        ls_index = []
        for train_index, test_index in self.split(X):
            ls_index.append((train_index, test_index))
        return ls_index
        
        
    def split(self, X, y=None, groups=None):
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        indices = np.arange(n_samples)
        
        
        if self.split_type == 'flat':
            if n_samples - self.n_splits - self.train_size - self.test_size + 1 < 0:
                raise ValueError("n_samples({0}) must be more than n_splits({1}) + train_size({3})+ test_size({2}) : " \
                                 .format(n_samples, self.n_splits, self.test_size, self.train_size))

            test_starts = range(n_samples - self.n_splits - self.test_size + 1 ,n_samples - self.test_size + 1,1)

            for test_start in test_starts:
                yield (indices[(test_start - self.train_size):test_start],
                       indices[test_start:test_start + self.test_size])
                
        if self.split_type  == 'with_min':
            """
            n_splits is changed to n_splits - min_train_size
            """
            if n_samples - self.n_splits < self.min_train_size:
                raise("n_samples({0}) - n_splits({1}) must be more than min_train_size({2})".format(n_samples, self.n_splits, self.min_train_size))

            test_starts = range(n_samples - self.n_splits - self.test_size + 1 ,n_samples - self.test_size + 1,1)
            test_starts = [ test_start for test_start in test_starts if test_start >= self.min_train_size ]
            
            
            for test_start in test_starts:
                yield (indices[:test_start],
                       indices[test_start:test_start + self.test_size])            
                    
tscv = customTimeSeriesSplit(n_splits=190, train_size=10, test_size=1, split_type = "with_min", min_train_size=15)


train_size is ignored


In [ ]:
for train_index, test_index in tscv.split(range(200,400)):
#     print(train_index, test_index)
    pass

In [314]:
Xy

,datetime,volumeRatio,code,dummyA,dummyB
385917,2019-01-01 09:00:00,0.602176,1987,0.897828,0.353214
385918,2019-01-01 09:30:00,0.491290,1987,0.832821,0.024793
385919,2019-01-01 10:00:00,0.939104,1987,0.577552,0.451552
385920,2019-01-01 10:30:00,0.403138,1987,0.798436,0.674146
385921,2019-01-01 11:00:00,0.053805,1987,0.007739,0.995179
...,...,...,...,...,...
386303,2019-01-30 13:30:00,0.938733,1987,0.543963,0.822097
386304,2019-01-30 14:00:00,0.001125,1987,0.367578,0.801516
386305,2019-01-30 14:30:00,0.549977,1987,0.450735,0.696127
386306,2019-01-30 15:00:00,0.053859,1987,0.449724,0.944015


In [320]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
grid_param ={'n_estimators':[1000],'max_depth':[4],'num_leaves':[15,3],'learning_rate':[0.1]}
#モデルの作成、各種ハイパラメータの指定

train_days = 100
test_days = 291
min_train_days = 20


Xy = df[df.code == 1987]
tscv = customTimeSeriesSplit(
    n_splits=test_days, train_size=train_days, test_size=1,  split_type = "flat")

for train_index, test_index in tscv.split(Xy):
    
    Xy_train_valid = Xy.iloc[train_index]
    Xy_test = Xy.iloc[test_index]
    
    Xy_train_
    
    
    
    bst = lgb.LGBMRegressor(
                            num_leaves = 31,
                            learning_rate=0.01,
                            min_child_samples=10,
                            n_estimators=1000,
                            max_depth=-1,
                            )
    fit_params={'early_stopping_rounds':10, 
                'eval_metric' : 'rmse', 
                'eval_set' : [(df[["dummyA","dummyB"]], df["volumeRatio"])]
               }

    cv_clf = GridSearchCV(
                bst, # 識別器
                grid_param, # 最適化したいパラメータセット 
                cv = customTimeSeriesSplit(
                    n_splits=len(train_index)-min_train_days, test_size=1, 
                    split_type = "with_min", min_train_size=min_train_days),
                scoring = 'neg_mean_squared_error',
                verbose = 0,
                n_jobs=3
                )

    cv_clf.fit(
                Xy_train[["dummyA","dummyB"]], 
                Xy_train["volumeRatio"],
                **fit_params,
                verbose = 0
                )
    
    y_pred = cv_clf.best_estimator_.predict(Xy_test[["dummyA","dummyB"]])

min_train_size is ignored
Fitting 80 folds for each of 2 candidates, totalling 160 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   14.0s
[Parallel(n_jobs=3)]: Done 160 out of 160 | elapsed:   31.6s finished


[1]	valid_0's rmse: 0.288758	valid_0's l2: 0.0833811
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.288977	valid_0's l2: 0.0835078
[3]	valid_0's rmse: 0.289278	valid_0's l2: 0.0836819
[4]	valid_0's rmse: 0.28936	valid_0's l2: 0.0837291
[5]	valid_0's rmse: 0.289607	valid_0's l2: 0.0838721
[6]	valid_0's rmse: 0.289895	valid_0's l2: 0.0840391
[7]	valid_0's rmse: 0.290281	valid_0's l2: 0.0842633
[8]	valid_0's rmse: 0.290359	valid_0's l2: 0.0843085
[9]	valid_0's rmse: 0.290404	valid_0's l2: 0.0843343
[10]	valid_0's rmse: 0.290599	valid_0's l2: 0.0844479
[11]	valid_0's rmse: 0.290685	valid_0's l2: 0.0844979
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.288758	valid_0's l2: 0.0833811
Fitting 80 folds for each of 2 candidates, totalling 160 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.0s
[Parallel(n_jobs=3)]: Done 160 out of 160 | elapsed:   23.6s finished


[1]	valid_0's rmse: 0.288876	valid_0's l2: 0.0834495
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.289177	valid_0's l2: 0.0836232
[3]	valid_0's rmse: 0.289611	valid_0's l2: 0.0838747
[4]	valid_0's rmse: 0.28992	valid_0's l2: 0.0840536
[5]	valid_0's rmse: 0.290598	valid_0's l2: 0.084447
[6]	valid_0's rmse: 0.29118	valid_0's l2: 0.0847856
[7]	valid_0's rmse: 0.291336	valid_0's l2: 0.0848767
[8]	valid_0's rmse: 0.292134	valid_0's l2: 0.0853425
[9]	valid_0's rmse: 0.292465	valid_0's l2: 0.0855356
[10]	valid_0's rmse: 0.292934	valid_0's l2: 0.0858101
[11]	valid_0's rmse: 0.293355	valid_0's l2: 0.0860573
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.288876	valid_0's l2: 0.0834495
Fitting 80 folds for each of 2 candidates, totalling 160 fits


...

[1]	valid_0's rmse: 0.288894	valid_0's l2: 0.0834599
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.289342	valid_0's l2: 0.0837187
[3]	valid_0's rmse: 0.290042	valid_0's l2: 0.0841243
[4]	valid_0's rmse: 0.290794	valid_0's l2: 0.0845614
[5]	valid_0's rmse: 0.291701	valid_0's l2: 0.0850892
[6]	valid_0's rmse: 0.292572	valid_0's l2: 0.0855984
[7]	valid_0's rmse: 0.292604	valid_0's l2: 0.0856173
[8]	valid_0's rmse: 0.292802	valid_0's l2: 0.085733
[9]	valid_0's rmse: 0.293759	valid_0's l2: 0.0862946
[10]	valid_0's rmse: 0.29398	valid_0's l2: 0.0864243
[11]	valid_0's rmse: 0.294346	valid_0's l2: 0.0866398
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.288894	valid_0's l2: 0.0834599
Fitting 80 folds for each of 2 candidates, totalling 160 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.8s
[Parallel(n_jobs=3)]: Done 160 out of 160 | elapsed:   23.2s finished


[1]	valid_0's rmse: 0.28906	valid_0's l2: 0.0835558
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 0.289401	valid_0's l2: 0.0837531
[3]	valid_0's rmse: 0.289865	valid_0's l2: 0.0840219
[4]	valid_0's rmse: 0.290002	valid_0's l2: 0.084101
[5]	valid_0's rmse: 0.290559	valid_0's l2: 0.0844246
[6]	valid_0's rmse: 0.291218	valid_0's l2: 0.0848077
[7]	valid_0's rmse: 0.291445	valid_0's l2: 0.0849402
[8]	valid_0's rmse: 0.291506	valid_0's l2: 0.0849759
[9]	valid_0's rmse: 0.29223	valid_0's l2: 0.0853984
[10]	valid_0's rmse: 0.292347	valid_0's l2: 0.0854667
[11]	valid_0's rmse: 0.292434	valid_0's l2: 0.0855175
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.28906	valid_0's l2: 0.0835558
Fitting 80 folds for each of 2 candidates, totalling 160 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    6.8s


KeyboardInterrupt: 

In [319]:
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.api import VAR  
import lightgbm as lgb
grid_param ={'n_estimators':[1000],'max_depth':[4],'num_leaves':[15,3],'learning_rate':[0.1]}
#モデルの作成、各種ハイパラメータの指定

train_days = 100
test_days = 291
min_train_days = 20
lag = 2

Xy = df[df.code == 1987]
tscv = customTimeSeriesSplit(
    n_splits=test_days, train_size=train_days, test_size=1,  split_type = "flat")

for train_index, test_index in tscv.split(Xy):
    
    Xy_train = Xy.iloc[train_index]
    Xy_test = Xy.iloc[test_index]
    
    model = VAR(data)
    results = model.fit(lag)#####
    results.forecast(Xy_train.values[-2:], 5)####
    
    y_pred = cv_clf.best_estimator_.predict(Xy_test[["dummyA","dummyB"]])

array([0.48948557])

In [321]:
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.api import VAR  
import statsmodels.formula.api as smf
import lightgbm as lgb
grid_param ={'n_estimators':[1000],'max_depth':[4],'num_leaves':[15,3],'learning_rate':[0.1]}
#モデルの作成、各種ハイパラメータの指定

train_days = 100
test_days = 291
min_train_days = 20
# lag = 2

Xy = df[df.code == 1987]
tscv = customTimeSeriesSplit(
    n_splits=test_days, train_size=train_days, test_size=1,  split_type = "flat")

######## ls_cols ########
for train_index, test_index in tscv.split(Xy):
    

    for 
    execVAR(Xy_vars,i_target):
    
    Xy_train = Xy.iloc[train_index]
    Xy_test = Xy.iloc[test_index]
    
    y_pred = execVAR(Xy_train)

min_train_size is ignored


NameError: name 'execVAR' is not defined

In [ ]:
lag = 2
def execVAR(Xy_vars,i_target):
    model = VAR(Xy_vars)
    results = model.fit(lag)#####
    pred = results.forecast(data.values[-2:],i_target)####
    return pred

def execMLR(X_train,y_train,X_test):
    model = sm.OLS(y_train, X_train)
    model = model.fit()
    pred = model.predict(X_test)
    return pred

In [ ]:
## ols も　var もなにのカラムをつかうかはvalidで証明している必要があるので、データをGBDT同様に分割